# Ordering


## Warning: This notebook will place live orders

Use a paper trading account (during market hours).


In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=13)
# util.logToConsole()

<IB connected to 127.0.0.1:7497 clientId=13>

Create a contract and a market order:

In [2]:
contract = Forex('EURUSD')
ib.qualifyContracts(contract)

order = LimitOrder('SELL', 20000, 1.11)

placeOrder will place the order order and return a ``Trade`` object right away (non-blocking):

In [3]:
trade = ib.placeOrder(contract, order)

``trade`` contains the order and everything related to it, such as order status, fills and a log.
It will be live updated with every status change or fill of the order.

In [4]:
ib.sleep(1)
trade.log

[TradeLogEntry(time=datetime.datetime(2022, 2, 7, 20, 45, 16, 940539, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2022, 2, 7, 20, 45, 17, 475233, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2022, 2, 7, 20, 45, 17, 605158, tzinfo=datetime.timezone.utc), status='Submitted', message='Fill 20000.0@1.14369', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2022, 2, 7, 20, 45, 17, 605158, tzinfo=datetime.timezone.utc), status='Filled', message='', errorCode=0)]

``trade`` will also available from ``ib.trades()``:

In [5]:
assert trade in ib.trades()

Likewise for ``order``:

In [6]:
assert order in ib.orders()

Now let's create a limit order with an unrealistic limit:

In [7]:
limitOrder = LimitOrder('BUY', 20000, 0.05)
limitTrade = ib.placeOrder(contract, limitOrder)

limitTrade

Trade(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), order=LimitOrder(orderId=5, clientId=13, action='BUY', totalQuantity=20000, lmtPrice=0.05), orderStatus=OrderStatus(orderId=5, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2022, 2, 7, 20, 46, 15, 937396, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)])

``status`` will change from "PendingSubmit" to "Submitted":

In [8]:
ib.sleep(1)
assert limitTrade.orderStatus.status == 'Submitted'

In [9]:
assert limitTrade in ib.openTrades()

Let's modify the limit price and resubmit:

In [10]:
limitOrder.lmtPrice = 0.10

ib.placeOrder(contract, limitOrder)

Trade(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), order=LimitOrder(orderId=5, clientId=13, permId=1179744878, action='BUY', totalQuantity=20000.0, lmtPrice=0.1, auxPrice=0.0), orderStatus=OrderStatus(orderId=5, status='Submitted', filled=0.0, remaining=20000.0, avgFillPrice=0.0, permId=1179744878, parentId=0, lastFillPrice=0.0, clientId=13, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2022, 2, 7, 20, 46, 15, 937396, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2022, 2, 7, 20, 46, 16, 134285, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2022, 2, 7, 20, 47, 33, 900793, tzinfo=datetime.timezone.utc), status='Submitted', message='Modify', errorCode=0)])

And now cancel it:

In [11]:
ib.cancelOrder(limitOrder)

Trade(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), order=LimitOrder(orderId=5, clientId=13, permId=1179744878, action='BUY', totalQuantity=20000.0, lmtPrice=0.1, auxPrice=0.0), orderStatus=OrderStatus(orderId=5, status='PendingCancel', filled=0.0, remaining=20000.0, avgFillPrice=0.0, permId=1179744878, parentId=0, lastFillPrice=0.0, clientId=13, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2022, 2, 7, 20, 46, 15, 937396, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2022, 2, 7, 20, 46, 16, 134285, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2022, 2, 7, 20, 47, 33, 900793, tzinfo=datetime.timezone.utc), status='Submitted', message='Modify', errorCode=0), TradeLogEntry(time=datetime.datetime(2022, 2, 7, 20, 47, 34, 40710, tzinfo=datetime.timezone.u

In [12]:
limitTrade.log

[TradeLogEntry(time=datetime.datetime(2022, 2, 7, 20, 46, 15, 937396, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2022, 2, 7, 20, 46, 16, 134285, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2022, 2, 7, 20, 47, 33, 900793, tzinfo=datetime.timezone.utc), status='Submitted', message='Modify', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2022, 2, 7, 20, 47, 34, 40710, tzinfo=datetime.timezone.utc), status='Submitted', message='Modified', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2022, 2, 7, 20, 47, 54, 189691, tzinfo=datetime.timezone.utc), status='PendingCancel', message='', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2022, 2, 7, 20, 47, 54, 337607, tzinfo=datetime.timezone.utc), status='Cancelled', message='', errorCode=0)]

placeOrder is not blocking and will not wait on what happens with the order.
To make the order placement blocking, that is to wait until the order is either
filled or canceled, consider the following:

In [13]:
%%time
order = MarketOrder('BUY', 100)

trade = ib.placeOrder(contract, order)
while not trade.isDone():
    ib.waitOnUpdate()

Wall time: 250 ms


What are our positions?

In [14]:
ib.positions()

[Position(account='DU5097959', contract=Forex('EURUSD', conId=12087792, localSymbol='EUR.USD', tradingClass='EUR.USD'), position=-19900.0, avgCost=1.1435899998918302)]

What's the total of commissions paid today?

In [15]:
sum(fill.commissionReport.commission for fill in ib.fills())

10.929895

whatIfOrder can be used to see the commission and the margin impact of an order without actually sending the order:

In [16]:
order = MarketOrder('SELL', 20000)
ib.whatIfOrder(contract, order)

OrderState(status='PreSubmitted', initMarginBefore='0.0', maintMarginBefore='0.0', equityWithLoanBefore='999687.04', initMarginChange='0.0', maintMarginChange='0.0', equityWithLoanChange='-1.7399999999906868', initMarginAfter='0.0', maintMarginAfter='0.0', equityWithLoanAfter='999685.3', commission=1.74662, minCommission=1.7976931348623157e+308, maxCommission=1.7976931348623157e+308, commissionCurrency='EUR', warningText='', completedTime='', completedStatus='')

In [17]:
ib.disconnect()